## predict fight outcome on current game state  
- current ults available per team
- teammates alive
- roles alive
- previous fights of map/round
- include map, some roles maybe more influential given map

UltsAvail = # ult rows where time in [kill1, kill2]
will have to go map by map, scrim by scrim

using primarily Kill, also UltimateCharged

### Notes:
- we can look at only attacker team win%, because defender is 1-a % win
- it's probably best to omit the identity of a player, so their performance relative to entered scrims does not skew

### collection steps:  
1. get kills data
2. number fights per map
3. add ults
4. re-index on time
5. calculate teammates alive
6. calculate n roles alive

In [1]:
import pandas as pd
import os
from dotenv import load_dotenv
import db_connection as db
import cleaning as cdb

load_dotenv(".env")
url = os.getenv("prod_url")

conn = db.createConnection(db.parseURL(url))

killQuery = """
SELECT "match_time",
    "event_type",
    "attacker_team",
    "attacker_hero",
    "attacker_name",
    "victim_team",
    "victim_hero",
    "victim_name",
    "MapDataId"
FROM "Kill"
"""

killDB = db.sqlSearch(conn, killQuery)
killDB.head()

,match_time,event_type,attacker_team,attacker_hero,attacker_name,victim_team,victim_hero,victim_name,MapDataId
0,19.73,kill,ASU,Junker Queen,Aurora,SCC,Kiriko,Reyzr,20.0
1,22.16,kill,SCC,Junker Queen,Tred,ASU,Sojourn,Jman,20.0
2,29.78,kill,ASU,Kiriko,zPanthr,SCC,Sojourn,Noxious,20.0
3,37.71,kill,ASU,Genji,Anhoo,SCC,Genji,Emmeryn,20.0
4,41.28,kill,ASU,Kiriko,zPanthr,SCC,Lúcio,redex,20.0


In [2]:
def calculate_fight_ids(group):
    group['time_prev'] = group['match_time'].diff()
    group['time_next'] = group['time_prev'].shift(-1)
    group['is_stagger'] = (group['time_prev'] > 15) & (group['time_next'] > 15)
    group['fight_id'] = group.loc[~group['is_stagger'], 'time_prev'].gt(15).cumsum() + 1
    group['fight_id'] = group['fight_id'].fillna(-1).astype(int)
    group = group.drop(['time_prev', 'time_next'], axis = 1)

    return group

killDB = killDB.groupby('MapDataId')
killDB = killDB.apply(calculate_fight_ids)
killDB.reset_index(inplace = True, drop = True)

killDB.head(5)


C:\Users\Xavier\AppData\Local\Temp\ipykernel_9940\65227948.py:12: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  killDB = killDB.apply(calculate_fight_ids)


,match_time,event_type,attacker_team,attacker_hero,attacker_name,victim_team,victim_hero,victim_name,MapDataId,is_stagger,fight_id
0,19.73,kill,ASU,Junker Queen,Aurora,SCC,Kiriko,Reyzr,20.0,False,1
1,22.16,kill,SCC,Junker Queen,Tred,ASU,Sojourn,Jman,20.0,False,1
2,29.78,kill,ASU,Kiriko,zPanthr,SCC,Sojourn,Noxious,20.0,False,1
3,37.71,kill,ASU,Genji,Anhoo,SCC,Genji,Emmeryn,20.0,False,1
4,41.28,kill,ASU,Kiriko,zPanthr,SCC,Lúcio,redex,20.0,False,1


get ult charges

In [3]:
ultQuery = """
SELECT "match_time",
    "event_type",
    "player_team",
    "player_hero",
    "player_name",
    "MapDataId"
FROM "UltimateCharged"
WHERE "MapDataId" IS NOT NULL
"""

ultDB = db.sqlSearch(conn, ultQuery)
ultDB.head(10)

,match_time,event_type,player_team,player_hero,player_name,MapDataId
0,91.24,ultimate_charged,ASU,Kiriko,zPanthr,20
1,97.55,ultimate_charged,SCC,Junker Queen,Tred,20
2,113.00,ultimate_charged,SCC,Lúcio,redex,20
3,127.47,ultimate_charged,SCC,Kiriko,Reyzr,20
4,130.83,ultimate_charged,ASU,Genji,Anhoo,20
5,140.38,ultimate_charged,SCC,Genji,Emmeryn,20
6,141.54,ultimate_charged,ASU,Junker Queen,Aurora,20
7,159.14,ultimate_charged,ASU,Sojourn,Jman,20
8,186.59,ultimate_charged,SCC,Sojourn,Noxious,20
9,211.71,ultimate_charged,ASU,Lúcio,Anghell1c,20


In [6]:
ultEndQuery = """
SELECT "match_time",
    "event_type",
    "player_team",
    "player_hero",
    "player_name",
    "MapDataId"
FROM "UltimateEnd"
WHERE "MapDataId" IS NOT NULL
"""

ultEnd = db.sqlSearch(conn, ultEndQuery)
ultEnd.head(10)

,match_time,event_type,player_team,player_hero,player_name,MapDataId
0,92.16,ultimate_end,ASU,Kiriko,zPanthr,20
1,102.51,ultimate_end,ASU,Kiriko,zPanthr,20
2,134.40,ultimate_end,SCC,Kiriko,Reyzr,20
3,145.23,ultimate_end,SCC,Kiriko,Reyzr,20
4,175.37,ultimate_end,ASU,Sojourn,Jman,20
5,179.60,ultimate_end,SCC,Lúcio,redex,20
6,182.54,ultimate_end,SCC,Genji,Emmeryn,20
7,209.40,ultimate_end,SCC,Sojourn,Noxious,20
8,214.40,ultimate_end,ASU,Genji,Anhoo,20
9,215.93,ultimate_end,ASU,Kiriko,zPanthr,20


In [11]:
combined = pd.concat([killDB, ultDB, ultEnd])
combined = combined.sort_values(by=['MapDataId','match_time'], ascending=[True, True])

combined.head(50)

,match_time,event_type,attacker_team,attacker_hero,attacker_name,victim_team,victim_hero,victim_name,MapDataId,is_stagger,fight_id,player_team,player_hero,player_name
0,19.73,kill,ASU,Junker Queen,Aurora,SCC,Kiriko,Reyzr,20.0,False,1.0,NaN,NaN,NaN
1,22.16,kill,SCC,Junker Queen,Tred,ASU,Sojourn,Jman,20.0,False,1.0,NaN,NaN,NaN
2,29.78,kill,ASU,Kiriko,zPanthr,SCC,Sojourn,Noxious,20.0,False,1.0,NaN,NaN,NaN
3,37.71,kill,ASU,Genji,Anhoo,SCC,Genji,Emmeryn,20.0,False,1.0,NaN,NaN,NaN
4,41.28,kill,ASU,Kiriko,zPanthr,SCC,Lúcio,redex,20.0,False,1.0,NaN,NaN,NaN
5,42.61,kill,SCC,Junker Queen,Tred,ASU,Lúcio,Anghell1c,20.0,False,1.0,NaN,NaN,NaN
6,86.24,kill,SCC,Junker Queen,Tred,ASU,Lúcio,Anghell1c,20.0,False,2.0,NaN,NaN,NaN
0,91.24,ultimate_charged,NaN,NaN,NaN,NaN,NaN,NaN,20.0,NaN,NaN,ASU,Kiriko,zPanthr
7,92.03,kill,ASU,Genji,Anhoo,SCC,Genji,Emmeryn,20.0,False,2.0,NaN,NaN,NaN
0,92.16,ultimate_end,NaN,NaN,NaN,NaN,NaN,NaN,20.0,NaN,NaN,ASU,Kiriko,zPanthr


In [12]:
combined['event_vals'] = combined['event_type'].map({'ultimate_charged': 1, 'ultimate_end': -1}).fillna(0)

# Calculate ultimates for attacker team
combined['attacker_ultimates'] = combined[combined['player_team'] == combined['attacker_team']].groupby('attacker_team')['event_type'].cumsum()
combined['attacker_ultimates'] = combined.groupby('attacker_team')['attacker_ultimates'].ffill().fillna(0)

# Calculate ultimates for victim team
combined['victim_ultimates'] = combined[combined['player_team'] == combined['victim_team']].groupby('victim_team')['event_type'].cumsum()
combined['victim_ultimates'] = combined.groupby('victim_team')['victim_ultimates'].ffill().fillna(0)

combined.head(20)

TypeError: cumsum is not supported for object dtype

: 